In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import re
# Load a grammar-checking model (Mistral, LLaMA, or GPT4All-based)
model_name = "meta-llama/Llama-3.2-1B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,legacy=False)
tokenizer.pad_token = tokenizer.eos_token
# Load model with mismatch tolerance
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map=0, 
    # load_in_4bit=True,
    # load_in_8bit=True,
    trust_remote_code=True,
    ignore_mismatched_sizes=True
)

# Create pipeline
pipe = pipeline("text-generation", 
                model=model, 
                tokenizer=tokenizer)
pipe.model = torch.compile(pipe.model)

def check_grammar(sentences):
    # Create a prompt for grammar correction for all sentences in batch
    prompts = [f"""Correct the following sentence for grammar and clarity. Only return the corrected sentence without explanations or additional text.
    Sentence: "{sent}"
    Corrected:""" for sent in sentences]
    
    # Process sentences in batch and immediately print results
    for response in pipe(prompts, max_length=1000, truncation=True, do_sample=False):
        corrected_text = response['generated_text'].strip()
        yield corrected_text

def extract_sentences(latex_file):
    with open(latex_file, 'r', encoding='utf-8') as f:
        text = f.read()

    # Remove LaTeX commands
    clean_text = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', text)  # Basic removal of commands
    clean_text = re.sub(r'\\[a-zA-Z]+\s?', '', clean_text)  # Remove standalone commands
    clean_text = re.sub(r'\$.*?\$', '', clean_text)  # Remove inline math
    clean_text = re.sub(r'\\begin\{.*?\}.*?\\end\{.*?\}', '', clean_text, flags=re.DOTALL)  # Remove environments

    # Split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', clean_text)
    return sentences

def process_latex_file(latex_file):
    sentences = extract_sentences(latex_file)
    for original, corrected in zip(sentences, check_grammar(sentences)):
        print(f"Original: {original}\nCorrected: {corrected}\n")

# Example usage
latex_file = "main.tex"
process_latex_file(latex_file)


Device set to use cuda:0


In [1]:
import asyncio
from ollama import AsyncClient

async def chat():
  message = {'role': 'user', 'content': 'Why is the sky blue?'}
  async for part in await AsyncClient().chat(model='deepseek-r1:7b', messages=[message], stream=True):
    print(part['message']['content'], end='', flush=True)

asyncio.run(chat())

RuntimeError: asyncio.run() cannot be called from a running event loop